In [69]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier


In [70]:

# Load the data for each standard
scores_5th = pd.read_csv("Datasets/Growth5th.csv")
scores_6th = pd.read_csv("Datasets/Growth6th.csv")
scores_7th = pd.read_csv("Datasets/Growth7th.csv")
scores_8th = pd.read_csv("Datasets/Growth8th.csv")
scores_9th = pd.read_csv("Datasets/Growth9th.csv")
scores_10th = pd.read_csv("Datasets/Growth10th.csv")

# Rename columns to include the year for easier merging
scores_5th = scores_5th.add_suffix("_5").rename(columns={"Unique_ID_5": "Unique_ID"})
scores_6th = scores_6th.add_suffix("_6").rename(columns={"Unique_ID_6": "Unique_ID"})
scores_7th = scores_7th.add_suffix("_7").rename(columns={"Unique_ID_7": "Unique_ID"})
scores_8th = scores_8th.add_suffix("_8").rename(columns={"Unique_ID_8": "Unique_ID"})
scores_9th = scores_9th.add_suffix("_9").rename(columns={"Unique_ID_9": "Unique_ID"})
scores_10th = scores_10th.add_suffix("_10").rename(columns={"Unique_ID_10": "Unique_ID"})

# Merge datasets on Unique_ID
df = scores_5th.merge(scores_6th, on="Unique_ID") \
               .merge(scores_7th, on="Unique_ID") \
               .merge(scores_8th, on="Unique_ID") \
               .merge(scores_9th, on="Unique_ID") \
               .merge(scores_10th, on="Unique_ID")
df

,Unique_ID,Marathi_5,Urdu_5,Hindi_5,English_5,History_5,Science_5,Geography_5,Drawing_5,Sports_5,...,Geography_10,Drawing_10,Sports_10,Environmental Studies_10,Algebra_10,Geometry_10,Computer_10,Defence_10,Sanskrit_10,Interest_10
0,stud_1,25,33,33,83,63,13,20,92,99,...,29,36,38,8,63,15,65,77,96,"Hindi, Science, Sanskrit"
1,stud_2,52,58,64,24,34,20,43,89,89,...,14,75,38,11,52,99,98,86,45,"Hindi, Geometry, Computer"
2,stud_3,76,24,94,29,91,10,28,86,52,...,47,1,51,87,87,31,42,28,9,"English, Environmental Studies, Algebra"
3,stud_4,41,70,81,98,74,13,40,56,25,...,23,80,22,78,92,44,78,85,73,"Algebra, Defence"
4,stud_5,88,54,49,78,2,11,90,80,89,...,50,35,72,64,28,15,36,37,2,"English, History"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,49,16,44,23,85,12,27,69,4,...,23,41,94,64,23,9,84,87,89,"Urdu, Sports"
9996,stud_9997,62,11,74,71,83,60,83,78,97,...,77,28,62,34,24,55,91,26,68,"Hindi, Computer"
9997,stud_9998,34,72,78,56,80,100,90,67,55,...,58,94,18,24,64,50,44,24,42,"English, Drawing"
9998,stud_9999,68,26,39,80,59,2,68,91,45,...,35,14,34,17,64,92,91,93,89,"Geometry, Defence"


In [71]:
df = df.drop(columns=["Interest_5", "Interest_6", "Interest_7", "Interest_8", "Interest_9", "Interest_10"])
df

,Unique_ID,Marathi_5,Urdu_5,Hindi_5,English_5,History_5,Science_5,Geography_5,Drawing_5,Sports_5,...,Science_10,Geography_10,Drawing_10,Sports_10,Environmental Studies_10,Algebra_10,Geometry_10,Computer_10,Defence_10,Sanskrit_10
0,stud_1,25,33,33,83,63,13,20,92,99,...,85,29,36,38,8,63,15,65,77,96
1,stud_2,52,58,64,24,34,20,43,89,89,...,44,14,75,38,11,52,99,98,86,45
2,stud_3,76,24,94,29,91,10,28,86,52,...,24,47,1,51,87,87,31,42,28,9
3,stud_4,41,70,81,98,74,13,40,56,25,...,83,23,80,22,78,92,44,78,85,73
4,stud_5,88,54,49,78,2,11,90,80,89,...,21,50,35,72,64,28,15,36,37,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,49,16,44,23,85,12,27,69,4,...,23,23,41,94,64,23,9,84,87,89
9996,stud_9997,62,11,74,71,83,60,83,78,97,...,56,77,28,62,34,24,55,91,26,68
9997,stud_9998,34,72,78,56,80,100,90,67,55,...,31,58,94,18,24,64,50,44,24,42
9998,stud_9999,68,26,39,80,59,2,68,91,45,...,70,35,14,34,17,64,92,91,93,89


In [72]:
# # Example subjects for 5th to 8th, and 9th & 10th
# subjects_5_to_8 = ["Marathi", "Urdu", "Hindi", "English", "History", "Science", 
#                    "Geography", "Drawing", "Sports", "Environmental Studies", "Math", "Computer"]

# subjects_9_to_10 = ["Algebra", "Geometry", "Defence", "Sanskrit"]

# # Combine all subjects
# all_subjects = subjects_5_to_8 + subjects_9_to_10

# # Assuming 'df' contains the original dataset with scores for 5th to 10th standard and Unique_ID
# # Add missing subjects (e.g., Algebra, Geometry, Defence, Sanskrit) to earlier years (5th to 8th)
# for subject in subjects_9_to_10:
#     for year in range(5, 9):  # Fill 5th to 8th with NaN for these subjects
#         df[f"{subject}_{year}"] = np.nan

# # Verify columns to check if NaN-filled subjects are added
# print("Columns after adding missing subjects:")
# print(df.columns)

In [90]:
# Growth 5th to 6th
# Example subjects for 5th to 8th
subjects_5_to_8 = ["Marathi", "Urdu", "Hindi", "English", "History", "Science", 
                   "Geography", "Drawing", "Sports", "Environmental Studies", "Math", "Computer"]
year = 5
next_year = 6
growth_5_to_6 = {"Unique_ID": df["Unique_ID"]}  # Start with Unique_ID
for subject in subjects_5_to_8:
    if f"{subject}_{year}" in df.columns and f"{subject}_{next_year}" in df.columns:
        condition = df[f"{subject}_{year}"].notna() & df[f"{subject}_{next_year}"].notna()
        growth_5_to_6[f"{subject}_Growth_{year}_to_{next_year}"] = np.where(
            condition, df[f"{subject}_{next_year}"] - df[f"{subject}_{year}"], np.nan
        )
    else:
        growth_5_to_6[f"{subject}_Growth_{year}_to_{next_year}"] = np.nan

growth_5_to_6_df = pd.DataFrame(growth_5_to_6)
#growth_5_to_6_df.to_csv("Growth_5_to_6.csv", index=False)
print("Growth 5th to 6th:")
#print(growth_5_to_6_df.head())
growth_5_to_6_df

Growth 5th to 6th:


,Unique_ID,Marathi_Growth_5_to_6,Urdu_Growth_5_to_6,Hindi_Growth_5_to_6,English_Growth_5_to_6,History_Growth_5_to_6,Science_Growth_5_to_6,Geography_Growth_5_to_6,Drawing_Growth_5_to_6,Sports_Growth_5_to_6,Environmental Studies_Growth_5_to_6,Math_Growth_5_to_6,Computer_Growth_5_to_6
0,stud_1,14.0,13.0,21.0,-48.0,8.0,51.0,54.0,-39.0,-71.0,-25.0,-25.0,-4.0
1,stud_2,2.0,-2.0,5.0,30.0,38.0,7.0,-21.0,-10.0,-42.0,59.0,9.0,-6.0
2,stud_3,-17.0,44.0,-72.0,48.0,-31.0,26.0,46.0,-15.0,3.0,65.0,64.0,-9.0
3,stud_4,-10.0,-9.0,-41.0,-73.0,-48.0,48.0,38.0,-34.0,13.0,40.0,-11.0,-4.0
4,stud_5,-36.0,-29.0,-27.0,-57.0,57.0,24.0,-46.0,-27.0,-27.0,8.0,-40.0,-15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,22.0,63.0,-17.0,0.0,-45.0,42.0,-7.0,-20.0,52.0,2.0,-56.0,-2.0
9996,stud_9997,-5.0,45.0,3.0,1.0,-60.0,-38.0,-40.0,-35.0,-72.0,-44.0,37.0,-1.0
9997,stud_9998,-12.0,3.0,-9.0,-23.0,-55.0,-26.0,-55.0,-23.0,25.0,-8.0,-37.0,17.0
9998,stud_9999,-42.0,-6.0,33.0,-22.0,-34.0,56.0,-22.0,-45.0,35.0,32.0,23.0,-21.0


In [91]:
year = 6
next_year = 7
growth_6_to_7 = {"Unique_ID": df["Unique_ID"]}  # Start with Unique_ID
for subject in subjects_5_to_8:
    if f"{subject}_{year}" in df.columns and f"{subject}_{next_year}" in df.columns:
        condition = df[f"{subject}_{year}"].notna() & df[f"{subject}_{next_year}"].notna()
        growth_6_to_7[f"{subject}_Growth_{year}_to_{next_year}"] = np.where(
            condition, df[f"{subject}_{next_year}"] - df[f"{subject}_{year}"], np.nan
        )
    else:
        growth_6_to_7[f"{subject}_Growth_{year}_to_{next_year}"] = np.nan

growth_5_to_6_df = pd.DataFrame(growth_5_to_6)
#growth_5_to_6_df.to_csv("Growth_6_to_7.csv", index=False)
print("Growth 6th to 7th:")
#print(growth_6_to_7_df.head())
growth_6_to_7_df

Growth 6th to 7th:


,Unique_ID,Marathi_Growth_6_to_7,Urdu_Growth_6_to_7,Hindi_Growth_6_to_7,English_Growth_6_to_7,History_Growth_6_to_7,Science_Growth_6_to_7,Geography_Growth_6_to_7,Drawing_Growth_6_to_7,Sports_Growth_6_to_7,Environmental Studies_Growth_6_to_7,Math_Growth_6_to_7,Computer_Growth_6_to_7
0,stud_1,35.0,0.0,20.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,stud_2,0.0,34.0,0.0,0.0,0.0,63.0,0.0,0.0,43.0,0.0,0.0,0.0
2,stud_3,0.0,0.0,0.0,5.0,0.0,46.0,0.0,0.0,0.0,0.0,6.0,29.0
3,stud_4,0.0,0.0,0.0,0.0,0.0,32.0,15.0,71.0,0.0,17.0,0.0,0.0
4,stud_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,stud_9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,stud_9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,stud_9999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
year = 7
next_year = 8
growth_7_to_8 = {"Unique_ID": df["Unique_ID"]}  # Start with Unique_ID
for subject in subjects_5_to_8:
    if f"{subject}_{year}" in df.columns and f"{subject}_{next_year}" in df.columns:
        condition = df[f"{subject}_{year}"].notna() & df[f"{subject}_{next_year}"].notna()
        growth_7_to_8[f"{subject}_Growth_{year}_to_{next_year}"] = np.where(
            condition, df[f"{subject}_{next_year}"] - df[f"{subject}_{year}"], np.nan
        )
    else:
        growth_7_to_8[f"{subject}_Growth_{year}_to_{next_year}"] = np.nan

growth_5_to_6_df = pd.DataFrame(growth_5_to_6)
#growth_5_to_6_df.to_csv("Growth_7_to_8.csv", index=False)
print("Growth 7th to 8th:")
#print(growth_7_to_8_df.head())
growth_7_to_8_df

Growth 7th to 8th:


,Unique_ID,Marathi_Growth_7_to_8,Urdu_Growth_7_to_8,Hindi_Growth_7_to_8,English_Growth_7_to_8,History_Growth_7_to_8,Science_Growth_7_to_8,Geography_Growth_7_to_8,Drawing_Growth_7_to_8,Sports_Growth_7_to_8,Environmental Studies_Growth_7_to_8,Math_Growth_7_to_8,Computer_Growth_7_to_8
0,stud_1,13.0,-32.0,-50.0,49.0,13.0,-49.0,-59.0,11.0,41.0,-19.0,-13.0,-7.0
1,stud_2,-51.0,8.0,1.0,-18.0,26.0,-36.0,11.0,-65.0,-13.0,-3.0,21.0,-11.0
2,stud_3,5.0,32.0,1.0,8.0,3.0,18.0,-40.0,-50.0,-39.0,-58.0,-44.0,6.0
3,stud_4,35.0,9.0,-16.0,66.0,-16.0,-75.0,-23.0,7.0,-19.0,7.0,41.0,11.0
4,stud_5,27.0,54.0,52.0,-8.0,-21.0,-4.0,17.0,30.0,-5.0,-24.0,10.0,-72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,-38.0,-48.0,26.0,32.0,43.0,-49.0,56.0,29.0,-6.0,-40.0,2.0,27.0
9996,stud_9997,9.0,22.0,-35.0,-3.0,36.0,-19.0,-34.0,-23.0,68.0,49.0,-30.0,23.0
9997,stud_9998,49.0,-6.0,-18.0,-22.0,58.0,22.0,34.0,-3.0,-72.0,68.0,-13.0,-42.0
9998,stud_9999,23.0,72.0,-71.0,32.0,30.0,29.0,28.0,46.0,-36.0,-8.0,-66.0,31.0


In [76]:
# Define the subjects present in both 8th and 9th standards
# Subjects present in both 8th and 9th (modify as needed)
subjects_8_to_9 = ["Marathi", "Urdu", "Hindi", "English", "History", "Science", 
                   "Geography", "Drawing", "Sports", "Environmental Studies", "Algebra", "Geometry", "Computer", "Defence"]

# Calculate Growth 8th to 9th
year = 8
next_year = 9
growth_8_to_9 = {"Unique_ID": df["Unique_ID"]}  # Start with Unique_ID

for subject in subjects_8_to_9:
    col_8 = f"{subject}_{year}"
    col_9 = f"{subject}_{next_year}"
    
    if col_8 in df.columns and col_9 in df.columns:
        # Only calculate growth where both values are not NaN
        condition = df[col_8].notna() & df[col_9].notna()
        growth_8_to_9[f"{subject}_Growth_{year}_to_{next_year}"] = np.where(
            condition, df[col_9] - df[col_8], np.nan
        )
    else:
        growth_8_to_9[f"{subject}_Growth_{year}_to_{next_year}"] = np.nan

# Create DataFrame for the growth calculations
growth_8_to_9_df = pd.DataFrame(growth_8_to_9)

# Debugging: Check the final DataFrame for growth
print("Growth 8th to 9th:")
growth_8_to_9_df

# Optionally, save the growth to CSV file
#growth_8_to_9_df.to_csv("Growth_8_to_9.csv", index=False)

Growth 8th to 9th:


,Unique_ID,Marathi_Growth_8_to_9,Urdu_Growth_8_to_9,Hindi_Growth_8_to_9,English_Growth_8_to_9,History_Growth_8_to_9,Science_Growth_8_to_9,Geography_Growth_8_to_9,Drawing_Growth_8_to_9,Sports_Growth_8_to_9,Environmental Studies_Growth_8_to_9,Algebra_Growth_8_to_9,Geometry_Growth_8_to_9,Computer_Growth_8_to_9,Defence_Growth_8_to_9
0,stud_1,-42.0,10.0,47.0,-64.0,-33.0,50.0,56.0,-12.0,-20.0,88.0,NaN,NaN,21.0,NaN
1,stud_2,85.0,-67.0,30.0,61.0,-5.0,-1.0,-22.0,-3.0,-54.0,24.0,NaN,NaN,43.0,NaN
2,stud_3,21.0,-46.0,52.0,-13.0,-54.0,-30.0,40.0,36.0,0.0,79.0,NaN,NaN,-3.0,NaN
3,stud_4,20.0,7.0,35.0,-51.0,37.0,47.0,-24.0,-2.0,79.0,-14.0,NaN,NaN,-24.0,NaN
4,stud_5,21.0,-75.0,-6.0,0.0,47.0,-15.0,7.0,-65.0,-41.0,29.0,NaN,NaN,41.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,6.0,62.0,-38.0,-2.0,-35.0,46.0,-73.0,-49.0,-26.0,81.0,NaN,NaN,-6.0,NaN
9996,stud_9997,9.0,8.0,-32.0,-51.0,34.0,7.0,-6.0,15.0,-66.0,-42.0,NaN,NaN,39.0,NaN
9997,stud_9998,-12.0,-61.0,-8.0,50.0,-7.0,-12.0,-12.0,56.0,36.0,-1.0,NaN,NaN,4.0,NaN
9998,stud_9999,31.0,-42.0,68.0,-57.0,-44.0,-83.0,-7.0,-81.0,-43.0,29.0,NaN,NaN,-74.0,NaN


In [83]:
# Define the subjects present in both 9th and 10th standards
# Modify as per your dataset
subjects_9_to_10 = ["Marathi", "Urdu", "Hindi", "English", "History", "Science", 
                    "Geography", "Drawing", "Sports", "Environmental Studies", "Algebra", 
                    "Geometry", "Computer", "Defence", "Sanskrit"]

# Calculate Growth 9th to 10th
year = 9
next_year = 10
growth_9_to_10 = {"Unique_ID": df["Unique_ID"]}  # Start with Unique_ID

for subject in subjects_9_to_10:
    col_9 = f"{subject}_{year}"
    col_10 = f"{subject}_{next_year}"
    
    if col_9 in df.columns and col_10 in df.columns:
        # Only calculate growth where both values are not NaN
        condition = df[col_9].notna() & df[col_10].notna()
        growth_9_to_10[f"{subject}_Growth_{year}_to_{next_year}"] = np.where(
            condition, df[col_10] - df[col_9], np.nan
        )
    else:
        growth_9_to_10[f"{subject}_Growth_{year}_to_{next_year}"] = np.nan

# Create DataFrame for the growth calculations
growth_9_to_10_df = pd.DataFrame(growth_9_to_10)

# Debugging: Check the final DataFrame for growth
print("Growth 9th to 10th:")
#print(growth_9_to_10_df.head())
growth_9_to_10_df

# Optionally, save the growth to CSV file
#growth_9_to_10_df.to_csv("Growth_9_to_10.csv", index=False)

Growth 9th to 10th:


,Unique_ID,Marathi_Growth_9_to_10,Urdu_Growth_9_to_10,Hindi_Growth_9_to_10,English_Growth_9_to_10,History_Growth_9_to_10,Science_Growth_9_to_10,Geography_Growth_9_to_10,Drawing_Growth_9_to_10,Sports_Growth_9_to_10,Environmental Studies_Growth_9_to_10,Algebra_Growth_9_to_10,Geometry_Growth_9_to_10,Computer_Growth_9_to_10,Defence_Growth_9_to_10,Sanskrit_Growth_9_to_10
0,stud_1,12.0,-9.0,25.0,59.0,-2.0,20.0,-42.0,-16.0,-11.0,-81.0,11.0,-78.0,-16.0,49.0,NaN
1,stud_2,-62.0,-8.0,-1.0,-22.0,-29.0,-9.0,3.0,64.0,15.0,-89.0,19.0,61.0,41.0,56.0,NaN
2,stud_3,-8.0,24.0,-30.0,14.0,68.0,-46.0,-27.0,-56.0,35.0,-13.0,-13.0,8.0,-43.0,-3.0,NaN
3,stud_4,-52.0,-19.0,-28.0,-19.0,4.0,18.0,-23.0,-18.0,-76.0,-8.0,85.0,-7.0,31.0,-2.0,NaN
4,stud_5,-62.0,17.0,-48.0,85.0,-10.0,5.0,-18.0,17.0,56.0,14.0,24.0,-81.0,-9.0,-23.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,-17.0,3.0,61.0,40.0,-36.0,-28.0,20.0,12.0,70.0,-36.0,-73.0,0.0,-10.0,19.0,NaN
9996,stud_9997,-74.0,-49.0,70.0,4.0,-34.0,46.0,74.0,-7.0,35.0,4.0,-67.0,43.0,8.0,-59.0,NaN
9997,stud_9998,-58.0,59.0,-2.0,34.0,-57.0,-53.0,1.0,-3.0,-26.0,-73.0,8.0,38.0,33.0,-61.0,NaN
9998,stud_9999,-50.0,-46.0,20.0,-1.0,78.0,66.0,-32.0,3.0,33.0,-39.0,17.0,46.0,83.0,18.0,NaN


In [79]:
growth_8_to_9_df["Algebra_Growth_8_to_9"] = growth_8_to_9_df["Algebra_Growth_8_to_9"].fillna(0)
growth_8_to_9_df

,Unique_ID,Marathi_Growth_8_to_9,Urdu_Growth_8_to_9,Hindi_Growth_8_to_9,English_Growth_8_to_9,History_Growth_8_to_9,Science_Growth_8_to_9,Geography_Growth_8_to_9,Drawing_Growth_8_to_9,Sports_Growth_8_to_9,Environmental Studies_Growth_8_to_9,Algebra_Growth_8_to_9,Geometry_Growth_8_to_9,Computer_Growth_8_to_9,Defence_Growth_8_to_9
0,stud_1,-42.0,10.0,47.0,-64.0,-33.0,50.0,56.0,-12.0,-20.0,88.0,0.0,NaN,21.0,NaN
1,stud_2,85.0,-67.0,30.0,61.0,-5.0,-1.0,-22.0,-3.0,-54.0,24.0,0.0,NaN,43.0,NaN
2,stud_3,21.0,-46.0,52.0,-13.0,-54.0,-30.0,40.0,36.0,0.0,79.0,0.0,NaN,-3.0,NaN
3,stud_4,20.0,7.0,35.0,-51.0,37.0,47.0,-24.0,-2.0,79.0,-14.0,0.0,NaN,-24.0,NaN
4,stud_5,21.0,-75.0,-6.0,0.0,47.0,-15.0,7.0,-65.0,-41.0,29.0,0.0,NaN,41.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,6.0,62.0,-38.0,-2.0,-35.0,46.0,-73.0,-49.0,-26.0,81.0,0.0,NaN,-6.0,NaN
9996,stud_9997,9.0,8.0,-32.0,-51.0,34.0,7.0,-6.0,15.0,-66.0,-42.0,0.0,NaN,39.0,NaN
9997,stud_9998,-12.0,-61.0,-8.0,50.0,-7.0,-12.0,-12.0,56.0,36.0,-1.0,0.0,NaN,4.0,NaN
9998,stud_9999,31.0,-42.0,68.0,-57.0,-44.0,-83.0,-7.0,-81.0,-43.0,29.0,0.0,NaN,-74.0,NaN


In [80]:
growth_8_to_9_df["Geometry_Growth_8_to_9"] = growth_8_to_9_df["Geometry_Growth_8_to_9"].fillna(0)
growth_8_to_9_df

,Unique_ID,Marathi_Growth_8_to_9,Urdu_Growth_8_to_9,Hindi_Growth_8_to_9,English_Growth_8_to_9,History_Growth_8_to_9,Science_Growth_8_to_9,Geography_Growth_8_to_9,Drawing_Growth_8_to_9,Sports_Growth_8_to_9,Environmental Studies_Growth_8_to_9,Algebra_Growth_8_to_9,Geometry_Growth_8_to_9,Computer_Growth_8_to_9,Defence_Growth_8_to_9
0,stud_1,-42.0,10.0,47.0,-64.0,-33.0,50.0,56.0,-12.0,-20.0,88.0,0.0,0.0,21.0,NaN
1,stud_2,85.0,-67.0,30.0,61.0,-5.0,-1.0,-22.0,-3.0,-54.0,24.0,0.0,0.0,43.0,NaN
2,stud_3,21.0,-46.0,52.0,-13.0,-54.0,-30.0,40.0,36.0,0.0,79.0,0.0,0.0,-3.0,NaN
3,stud_4,20.0,7.0,35.0,-51.0,37.0,47.0,-24.0,-2.0,79.0,-14.0,0.0,0.0,-24.0,NaN
4,stud_5,21.0,-75.0,-6.0,0.0,47.0,-15.0,7.0,-65.0,-41.0,29.0,0.0,0.0,41.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,6.0,62.0,-38.0,-2.0,-35.0,46.0,-73.0,-49.0,-26.0,81.0,0.0,0.0,-6.0,NaN
9996,stud_9997,9.0,8.0,-32.0,-51.0,34.0,7.0,-6.0,15.0,-66.0,-42.0,0.0,0.0,39.0,NaN
9997,stud_9998,-12.0,-61.0,-8.0,50.0,-7.0,-12.0,-12.0,56.0,36.0,-1.0,0.0,0.0,4.0,NaN
9998,stud_9999,31.0,-42.0,68.0,-57.0,-44.0,-83.0,-7.0,-81.0,-43.0,29.0,0.0,0.0,-74.0,NaN


In [81]:
growth_8_to_9_df["Defence_Growth_8_to_9"] = growth_8_to_9_df["Defence_Growth_8_to_9"].fillna(0)
growth_8_to_9_df

,Unique_ID,Marathi_Growth_8_to_9,Urdu_Growth_8_to_9,Hindi_Growth_8_to_9,English_Growth_8_to_9,History_Growth_8_to_9,Science_Growth_8_to_9,Geography_Growth_8_to_9,Drawing_Growth_8_to_9,Sports_Growth_8_to_9,Environmental Studies_Growth_8_to_9,Algebra_Growth_8_to_9,Geometry_Growth_8_to_9,Computer_Growth_8_to_9,Defence_Growth_8_to_9
0,stud_1,-42.0,10.0,47.0,-64.0,-33.0,50.0,56.0,-12.0,-20.0,88.0,0.0,0.0,21.0,0.0
1,stud_2,85.0,-67.0,30.0,61.0,-5.0,-1.0,-22.0,-3.0,-54.0,24.0,0.0,0.0,43.0,0.0
2,stud_3,21.0,-46.0,52.0,-13.0,-54.0,-30.0,40.0,36.0,0.0,79.0,0.0,0.0,-3.0,0.0
3,stud_4,20.0,7.0,35.0,-51.0,37.0,47.0,-24.0,-2.0,79.0,-14.0,0.0,0.0,-24.0,0.0
4,stud_5,21.0,-75.0,-6.0,0.0,47.0,-15.0,7.0,-65.0,-41.0,29.0,0.0,0.0,41.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,6.0,62.0,-38.0,-2.0,-35.0,46.0,-73.0,-49.0,-26.0,81.0,0.0,0.0,-6.0,0.0
9996,stud_9997,9.0,8.0,-32.0,-51.0,34.0,7.0,-6.0,15.0,-66.0,-42.0,0.0,0.0,39.0,0.0
9997,stud_9998,-12.0,-61.0,-8.0,50.0,-7.0,-12.0,-12.0,56.0,36.0,-1.0,0.0,0.0,4.0,0.0
9998,stud_9999,31.0,-42.0,68.0,-57.0,-44.0,-83.0,-7.0,-81.0,-43.0,29.0,0.0,0.0,-74.0,0.0


In [87]:
growth_9_to_10_df = growth_9_to_10_df.drop(columns=["Sanskrit_Growth_9_to_10"], axis=1)
growth_9_to_10_df

,Unique_ID,Marathi_Growth_9_to_10,Urdu_Growth_9_to_10,Hindi_Growth_9_to_10,English_Growth_9_to_10,History_Growth_9_to_10,Science_Growth_9_to_10,Geography_Growth_9_to_10,Drawing_Growth_9_to_10,Sports_Growth_9_to_10,Environmental Studies_Growth_9_to_10,Algebra_Growth_9_to_10,Geometry_Growth_9_to_10,Computer_Growth_9_to_10,Defence_Growth_9_to_10
0,stud_1,12.0,-9.0,25.0,59.0,-2.0,20.0,-42.0,-16.0,-11.0,-81.0,11.0,-78.0,-16.0,49.0
1,stud_2,-62.0,-8.0,-1.0,-22.0,-29.0,-9.0,3.0,64.0,15.0,-89.0,19.0,61.0,41.0,56.0
2,stud_3,-8.0,24.0,-30.0,14.0,68.0,-46.0,-27.0,-56.0,35.0,-13.0,-13.0,8.0,-43.0,-3.0
3,stud_4,-52.0,-19.0,-28.0,-19.0,4.0,18.0,-23.0,-18.0,-76.0,-8.0,85.0,-7.0,31.0,-2.0
4,stud_5,-62.0,17.0,-48.0,85.0,-10.0,5.0,-18.0,17.0,56.0,14.0,24.0,-81.0,-9.0,-23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,-17.0,3.0,61.0,40.0,-36.0,-28.0,20.0,12.0,70.0,-36.0,-73.0,0.0,-10.0,19.0
9996,stud_9997,-74.0,-49.0,70.0,4.0,-34.0,46.0,74.0,-7.0,35.0,4.0,-67.0,43.0,8.0,-59.0
9997,stud_9998,-58.0,59.0,-2.0,34.0,-57.0,-53.0,1.0,-3.0,-26.0,-73.0,8.0,38.0,33.0,-61.0
9998,stud_9999,-50.0,-46.0,20.0,-1.0,78.0,66.0,-32.0,3.0,33.0,-39.0,17.0,46.0,83.0,18.0


In [94]:
# Merge all growth DataFrames on Unique_ID
overall_growth_df = growth_5_to_6_df.merge(growth_6_to_7_df, on="Unique_ID", how="outer") \
                                    .merge(growth_7_to_8_df, on="Unique_ID", how="outer") \
                                    .merge(growth_8_to_9_df, on="Unique_ID", how="outer") \
                                    .merge(growth_9_to_10_df, on="Unique_ID", how="outer")

# Fill NaN values with 0 for all growth columns
overall_growth_df = overall_growth_df.fillna(0)

# Initialize a dictionary to store overall growth rates
overall_growth = {"Unique_ID": overall_growth_df["Unique_ID"]}

# List of general subjects across multiple years
general_subjects = ["Marathi", "Urdu", "Hindi", "English", "History", "Science", 
                    "Geography", "Drawing", "Sports", "Environmental Studies", "Math", 
                    "Computer"]

# Calculate the overall growth rate for general subjects
for subject in general_subjects:
    # Collect all growth columns for the subject
    growth_columns = [col for col in overall_growth_df.columns if subject in col and "Growth" in col]
    
    # Calculate the mean growth for the subject across all years
    overall_growth[f"{subject}_Overall_Growth"] = overall_growth_df[growth_columns].mean(axis=1)

# Special handling for Algebra, Geometry, and Defence
special_subjects = ["Algebra", "Geometry", "Defence"]

for subject in special_subjects:
    # Collect growth columns for the subject
    growth_columns = [col for col in overall_growth_df.columns if subject in col and "Growth" in col]
    
    # Calculate overall growth as sum divided by 1 (since they exist in 9th to 10th only)
    overall_growth[f"{subject}_Overall_Growth"] = overall_growth_df[growth_columns].sum(axis=1)

# Convert to DataFrame
overall_growth_df = pd.DataFrame(overall_growth)

# Save the result to a CSV file
#overall_growth_df.to_csv("Overall_Growth_5_to_10.csv", index=False)

# Display the result
print("Overall Growth Rates:")
#print(overall_growth_df.head())
overall_growth_df


Overall Growth Rates:


,Unique_ID,Marathi_Overall_Growth,Urdu_Overall_Growth,Hindi_Overall_Growth,English_Overall_Growth,History_Overall_Growth,Science_Overall_Growth,Geography_Overall_Growth,Drawing_Overall_Growth,Sports_Overall_Growth,Environmental Studies_Overall_Growth,Math_Overall_Growth,Computer_Overall_Growth,Algebra_Overall_Growth,Geometry_Overall_Growth,Defence_Overall_Growth
0,stud_1,6.4,-3.6,12.6,-0.8,-2.2,14.4,1.8,-11.2,-12.2,-7.4,-12.666667,-1.2,11.0,-78.0,49.0
1,stud_2,-5.2,-7.0,7.0,10.2,6.0,4.8,-5.8,-2.8,-10.2,-1.8,10.000000,13.4,19.0,61.0,56.0
2,stud_3,0.2,10.8,-9.8,12.4,-2.8,2.8,3.8,-17.0,-0.2,14.6,8.666667,-4.0,-13.0,8.0,-3.0
3,stud_4,-1.4,-2.4,-10.0,-15.4,-4.6,14.0,-3.4,4.8,-0.6,8.4,10.000000,2.8,85.0,-7.0,-2.0
4,stud_5,-10.0,-6.6,-5.8,4.0,14.6,2.0,-8.0,-9.0,-3.4,5.4,-10.000000,-11.0,24.0,-81.0,-23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,stud_9996,-5.4,16.0,6.4,14.0,-14.6,2.2,-0.8,-5.6,18.0,1.4,-18.000000,1.8,-73.0,0.0,19.0
9996,stud_9997,-12.2,5.2,1.2,-9.8,-4.8,-0.8,-1.2,-10.0,-7.0,-6.6,2.333333,13.8,-67.0,43.0,-59.0
9997,stud_9998,-6.6,-1.0,-7.4,7.8,-12.2,-13.8,-6.4,5.4,-7.4,-2.8,-16.666667,2.4,8.0,38.0,-61.0
9998,stud_9999,-7.6,-4.4,10.0,-9.6,6.0,13.6,-6.6,-15.4,-2.2,2.8,-14.333333,3.8,17.0,46.0,18.0


stud_1 32.32% Inclining Marathi
stud_1 -18.18% Declining Urdu
stud_1 63.64% Inclining Hindi
stud_1 -4.04% Declining English
stud_1 -11.11% Declining History
stud_1 72.73% Inclining Science
stud_1 9.18% Inclining Geography
stud_1 -56.57% Declining Drawing
stud_1 -61.62% Declining Sports
stud_1 -37.37% Declining Environmental Studies
stud_1 -38.38% Declining Math
stud_1 -6.12% Declining Computer
stud_1 11.11% Inclining Algebra
stud_1 -78.79% Declining Geometry
stud_1 49.49% Inclining Defence
